    #VARIAÇÃO DO PREÇO DOS COMBUSTÍVEIS NA PARAÍBA ENTRE 2004 E 2019#
    Estudo do banco de dados de preços de combustíveis fósseis no Brasil a partir do banco de dados da Agência Nacional de Petróleo.
    A partir dos banco de informações fornecidas no arquivo "2004-2019.tsv" que fornece diversos dados a respeito dos preços máximo, preços mínimos, preços médios tanto de distribuição quanto de revenda. Além ainda medidas de dispersão a respeito desses dados.
    Esse relatório propõe-se a fazer uma análise visual dos preços médios do Etanol Hidratado e da Gasolina Comum nos postos de abastecimento do estado da Paraíba. 
    Foram postos em um gráfico a variação dos preços semanalmente e é destacado em vermelho os períodos em que é mais vantajoso abastecer o veículo com álcool, caso o mesmo seja motor flex.

In [2]:
import pandas as pd 
import numpy as np
import plotly           #(version 4.5.0)
import plotly.io as pio
import plotly.graph_objects as go
import os
import scipy.misc
from moviepy.editor import *
from glob import glob



Acima estão destacadas as bibliotecas utilizadas no projeto. As mais utilizadas serão a pandas para extrair e organizar os dados da maneira que desejamos e ainda a plotly que irá exportar a plotagem dos mesmos.

In [63]:
combustiveis = pd.read_csv("2004-2019.tsv", sep="\t")
#Extração dos dados do arquivo csv
postos_pb = combustiveis.loc[combustiveis['ESTADO']=='PARAIBA']
#Separação dos dados referentes ao estado da Paraíba
postos_pb['DIA'] = postos_pb['DATA INICIAL'].astype(str).str[-2:]
postos_pb['DIA'] = pd.to_numeric(postos_pb['DIA'].loc[:],errors='coerce')

'''Criação da coluna "DIA" que servirá de referência para a ordenação cronológica. Foi escolhida a data inicial 
para tal pois a mesma sempre acontece no mesmo mês. Enquanto a data final de pesquisa eventualmente pode encerrar 
no mês seguinte.'''

postos_pb = postos_pb.drop(['Unnamed: 0','DATA INICIAL',  'REGIÃO', 'ESTADO','MARGEM MÉDIA REVENDA',
                           'PREÇO MÉDIO DISTRIBUIÇÃO','DESVIO PADRÃO DISTRIBUIÇÃO','PREÇO MÍNIMO DISTRIBUIÇÃO','PREÇO MÁXIMO DISTRIBUIÇÃO','COEF DE VARIAÇÃO DISTRIBUIÇÃO'], axis=1)
#Excluidos os dados de colunas que não serão observadas na atual análise 
cols = postos_pb.columns.tolist()
cols = [cols[-1], cols[-3], cols[-2]] + cols[0:-3]
postos_pb = postos_pb[cols]
postos_pb = postos_pb.dropna()
#Reordenação das colunas e exclusão de linhas que venham a ter dados do tipo NaN.

postos_pb.head()
#Observação do dataset em suas 5 primeiras linhas.

C:\anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DIA,MÊS,ANO,DATA FINAL,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,COEF DE VARIAÇÃO REVENDA
8,9,5,2004,2004-05-15,ETANOL HIDRATADO,125,R$/l,1.284,0.130,1.100,1.699,0.101
35,16,5,2004,2004-05-22,ETANOL HIDRATADO,125,R$/l,1.284,0.134,1.090,1.699,0.104
62,23,5,2004,2004-05-29,ETANOL HIDRATADO,126,R$/l,1.283,0.127,1.099,1.699,0.099
89,30,5,2004,2004-06-05,ETANOL HIDRATADO,127,R$/l,1.283,0.129,1.120,1.699,0.101
116,6,6,2004,2004-06-12,ETANOL HIDRATADO,128,R$/l,1.300,0.116,1.140,1.699,0.089


In [ ]:
glp_pb = postos_pb[postos_pb['PRODUTO'] == 'GLP']
gnv_pb = postos_pb[postos_pb['PRODUTO'] == 'GNV']
diesels10_pb = postos_pb[postos_pb['PRODUTO'] == 'ÓLEO DIESEL S10']
diesel_pb = postos_pb[postos_pb['PRODUTO'] == 'ÓLEO DIESEL']

#Dataframe para cada produto em um dataframe específico para futuras análises. 
#Por enquanto estes não estão no escopo deste trabalho.

In [52]:
etanol_pb = postos_pb[postos_pb['PRODUTO'] == 'ETANOL HIDRATADO']
etanol_pb = etanol_pb.sort_values(by=['DATA FINAL'])
gasolina_pb = postos_pb[postos_pb['PRODUTO'] == 'GASOLINA COMUM']
gasolina_pb = gasolina_pb.sort_values(by=['DATA FINAL'])
#Acima temos a separação das análises para cada combustível pesquisado. Utiliza-se da função 'sort_values' pela coluna
#'DATA FINAL' para podermos saber que comparamos os preços do mesmo dia e criar a coluna 'color' que será responsável pela cor de preenchimento na plotagem.

etanol_pb['color'] = ['#CC2936' if (x < 0.7*y) else '#5A3A31' for x,y in zip(etanol_pb['PREÇO MÉDIO REVENDA'].loc[:],gasolina_pb['PREÇO MÉDIO REVENDA'].loc[:])]
#Cores de preenchimento para saber se compensa abastecer com etanol ou gasolina. 
#Caso o preço do etanol seja superior a 70% ao preço da gasolina compensará abastecer com gasolina.
# Quando for compensador abastecer com etanol o preenchimento será vermelho, caso contrário, marrom.

gasolina_pb['color'] = '#0267C1'
#preenchimento da gasolina será em azul, para todos os casos.

alc_gasolina = pd.concat([etanol_pb,gasolina_pb])
#criação do dataframe que concatena etanol e gasolina. A partir daqui as análises serão feitas somente nesse dataframe.




In [64]:
anos = [i for i in range(2004,2020)]
meses = [i for i in range(1,13)]
n_frames = {}
for ano in anos:
    for mes in meses:
        try:
            df = alc_gasolina[(alc_gasolina['MÊS'] == mes) & (alc_gasolina['ANO'] == ano)]
            df = df.sort_values(by=['DIA'])
            n_frames['Preço Combustíveis na Paraíba em ' + str(df['MÊS'].iloc[0]) + '/' + str(df['ANO'].iloc[0])] = df
        except:
            continue
'''
Criado o dicionário n_frames que será atráves dele que poderemos criar cada gráfico semanal e a partir disso criar o vídeo.
O dataframe df é criado para cada mês de um determinado ano. As funções try e except são utilizadas para caso de anos 
que não tenham todos os meses de análise, como 2004 e 2019.
'''

In [55]:
j = 0
for key, value in n_frames.items():
    for i in range (0,value.shape[0],2):
        fig = go.Figure(
            data=[
                go.Bar(x=value['PRODUTO'].iloc[i:i+2], y=np.asarray(value['PREÇO MÉDIO REVENDA'].iloc[i:i+2]),
                text=value['PREÇO MÉDIO REVENDA'].iloc[i:i+2], texttemplate='%{text:.4s}',
                textfont={'size':14}, textposition='inside', insidetextanchor='middle',
                width=0.9, marker={'color':value['color'].iloc[i:i+2]})
            ],
            layout=go.Layout(
            xaxis=dict(range=[-1, 2], autorange=False, title=dict(text='PRODUTO',font=dict(size=14))),
            yaxis=dict(range=[0, 5], autorange=False,tickfont=dict(size=14), title=dict(text='R$/LITRO',font=dict(size=14))),
            title=dict(text='Preço Combustíveis na Paraíba em ' + str(value['DIA'].iloc[i]) + '/' 
                   + str(value['MÊS'].iloc[i]) + '/' + str(value['ANO'].iloc[i]),
                   font=dict(size=28),x=0.5,xanchor='center'),
            # Add button
            )
        )
        if not os.path.exists("images_postos"):
            os.mkdir("images_postos")
        fig.write_image("images_postos/" + str(j) + ".png")
        j = j + 1

#Os dois laços 'for' percorrem, respectivamente, o dicionário criado no item anterior e o dataframe que está em value no dicionário
#Cada figura que é criada fica na pasta "images_postos" começando por 0 até a última semana do último dataframe.


182


In [2]:
images = ['images_postos/' + str(i) + '.png' for i in range(0,785)]
audio = AudioFileClip(<<MÚSICA DE SUA ESCOLHA EM FORMATO MP3>> + '.mp3')

clips = [ImageClip(m).set_duration(0.5)
      for m in images]

concat_clip = concatenate_videoclips(clips, method="compose")
new_audioclip = CompositeAudioClip([audio])

concat_clip.audio = new_audioclip
concat_clip.write_videofile("combustiveis.mp4", fps=24)

'''Criação do video para ser exportado com os frames criados. Concatenando cada frame com a mú'''

SyntaxError: invalid syntax (<ipython-input-2-830b8c3eeec9>, line 7)